#  Jogo dos Rastros
## Projeto nº 2
### Introdução à Inteligência Artificial edição 2020/21


## Grupo: 42

### Elementos do Grupo

Nome: Ivo Veiga

Número: 44865

Nome: João Silva

Número: 48782

Nome: Manuel Tovar

Número: 49522

### Explicação da função e sua lógica

A função tem três partes que avaliam um estado.

A primeira determina se é garantida uma vitória ou derrota, consoante a posição da peça branca. Estando esta na casa objetivo do adversário é considerado derrota, na casa objetivo do jogador é vitória. Se estiver a uma distancia = 1 (incuindo diagonais) do objetivo do jogador é considerado vitória pois se o jogo não se encontra num estado terminal a função avalia sempre um estado em que é o jogador a jogar. Se não for possível efetuar uma jogada então o estado é terminal e sendo assim é considerada vitória se for o oponente a jogar e derrota se for o jogador a jogar. Se for vitória é atribuido o valor 3 e derrota o valor -3. Este valores são o máximo e o minimo que a função pode devolver e são os valores que têm maior importância.

A segunda parte determina se a peça branca está demasiado perto da casa objetivo do adversário. É considerado demasiado perto se a distancia entre as duas for <= 1 . Com vários testes chegou-se à conclusão que sem esta verificação era mais comum o jogador fazer jogadas que mais á frente no jogo levássem a estados perigosos onde o oponente está a uma jogada da vitória e não há margem de manobra para o jogador escapar. Se for determinado que estiver demasiado perto, a função atribui ao estado o valor -1.5 . Este valor foi escolhido pois é maior que o -3 dado em caso de certeza de derrota e menor que os valores atribuidos na terceira parte que são considerados menos importantes.

A terceira parte é a principal. É avaliado um estado consoante o impacto das as jogadas que um oponente pode efetuar após uma jogada do jogador. É mais favorável se a jogada do oponente diminuir a distância entre a peça branca e a casa objetivo do jogador e menos favorável caso contrário. Isto é avaliado tendo em conta todas as jogadas que um jogador pode efetuar num dado estado e todas as jogadas que um oponente pode efetuar após a jogada do jogador. As jogadas do jogador são as casas livres ao redor da peça branca e as do oponente são as casas livres ao redor de uma casa onde o jogador pode jogar. É possível considerar casas livres mais de uma vez se forem casas também adjacentes a outra casa onde o jogador pode jogar. Ou seja, se o jogador jogar (4,5)->(5,5) o oponente pode jogar (5,5)->(6,5), se o jogador jogar (4,5)->(5,4) o oponente pode ir para na mesma para (6,5). Se uma jogada do oponente for favorável essa jogada vale 1. Se for desfavorável vale -1. Se for neutra vale 0. A avaliação final é a média dos valores de cada jogada e fica entre -1 e 1.

Aliando esta parte à maneira como o alpha-beta funciona em que a função de avaliação avalia um estado X jogadas mais à frente, tem-se que são favorecidos estados com configurações de peças pretas que impedem o oponente de afastar a peça branca da casa objetivo do jogador. Visto de outro modo, são mais favorecidas configurações com mais casas livres entre a casa objetivo do jogador e a peça branca e menos favorecidas as casas livres entre a casa objetivo do oponente e peça branca.

Isto manifesta-se num comportamento que procura estados com mais peças pretas entre a casa branca e a casa objetivo do oponente. Em termos práticos tem-se que logo no inicio o jogador procura bloquear a casa objetivo do oponente e ao longo do jogo procura encher o tabuleiro de peças começando da casa objetivo do oponente até chegar à do jogador. Assim, o jogador procura jogar afastado da sua casa objetivo para evitar que esta fique bloqueada.

A função procura evitar configurações de peças pretas que "encurralem" a peça branca, mas só para espaços pequenos (considera ao todo as casas num raio <= 2 da peça branca), mantendo aberta a zona na direção dacasa objetivo do jogador.

Explicações da implementação em comentário no código a seguir.

In [1]:
from rastros import *

#devolve a distancia tendo em conta a possibilidade de deslocação nas diagonais
def distancia_42 (a, b):
    return max(abs(a[0]-b[0]), abs(a[1]-b[1]))

#devolve as casas livres em volta de um ponto
def moves_from_point_42(point, state):
        alladjacent = [(point[0]+a, point[1]+b) for a in [-1,0,1] for b in [-1,0,1]]
        return [p for p in alladjacent
                if p not in state.blacks and p !=state.white and p !=point and p in state.fullboard]

#avalia a favorabilidade de um estado em que seja o turno do jogador
#é mais favovável se impedir o oponente de aumentar a distancia entre a peça branca e a casa objetivo do jogador
def general_eval_42(moves, state, goal):
    #vai ser retornada uma média dos pontos atribuidos a cada jogada possível
    move_score = 0
    num_moves = 0
    for move in moves: #jogadas que o jogador pode fazer
        for sub_move in moves_from_point_42(move, state): #jogadas que o oponente pode fazer
            #em comparação com uma jogada que o jogador pode fazer:
            #se uma jogada do oponente levar a peça branca para mais perto da casa objetivo do jogador: +1
            #se levar para mais longe: -1
            #se mantiver a distancia: 0
            s_score = distancia_42(move, goal) - distancia_42(sub_move,goal)
            move_score += s_score
            num_moves += 1
    if num_moves == 0:
        #se não houve jogadas do oponente o estado é terminal com vitória para o jogador
        #só funciona se esta len(moves) > 0 , é preciso verificar isto antes de chamar a função
        return 3
    return (move_score / num_moves)


def fun_aval_42(state, player):
    #determina o ponto onde player quer chegar
    goal = (8, 1)
    no_goal = (1, 8)
    if player == "N":
        goal = (1, 8)
        no_goal = (8, 1)
    
    #avalia se, pela posiçao da peça branca, o jogo terminaría
    #caso vitória atribui pontuação 3, caso derrota atribui -3
    if state.white == goal:
        return 3    
    if state.white == no_goal:
        return -3    
    if distancia_42(state.white, goal) <= 1:
        return 3    
    moves = state.moves()
    if len(moves) == 0:
        return -3 if state.to_move == player else 3
    
    #avalia se a peça branca está demasiado perto da casa final do adversário
    if distancia_42(state.white, no_goal) <= 1:
        return -1.5
    
    #por defeito atribui a pontuação determinada por general_eval()
    return general_eval_42(moves, state, goal)

### Demonstração

Exemplo de jogo entre basilio e jogador42:

In [2]:
jogador42 = Jogador("jogador42",
                  lambda game, state:
                  alphabeta_cutoff_search_new(state,game,depth_for_all,eval_fn=fun_aval_42))

jogo = jogaRastros11(basilio,jogador42)
mostraJogo(jogo[0], verbose = True)

Tabuleiro:
 12345678
1........1
2........2
3........3
4....o...4
5........5
6........6
7........7
8........8
 12345678 
Próximo jogador:S

Basilio-->  (5, 4)
Tabuleiro:
 12345678
1........1
2........2
3........3
4....@...4
5...o....5
6........6
7........7
8........8
 12345678 
Próximo jogador:N

jogador42-->  (6, 3)
Tabuleiro:
 12345678
1........1
2........2
3........3
4....@...4
5...@....5
6..o.....6
7........7
8........8
 12345678 
Próximo jogador:S

Basilio-->  (7, 2)
Tabuleiro:
 12345678
1........1
2........2
3........3
4....@...4
5...@....5
6..@.....6
7.o......7
8........8
 12345678 
Próximo jogador:N

jogador42-->  (6, 2)
Tabuleiro:
 12345678
1........1
2........2
3........3
4....@...4
5...@....5
6.o@.....6
7.@......7
8........8
 12345678 
Próximo jogador:S

Basilio-->  (7, 1)
Tabuleiro:
 12345678
1........1
2........2
3........3
4....@...4
5...@....5
6.@@.....6
7o@......7
8........8
 12345678 
Próximo jogador:N

jogador42-->  (6, 1)
Tabuleiro:
 12345678
1........1
2........2
3..

Exemplo de 3 campeonatos, mostrando os resultados agregados (é possível visualizar os jogos todos com "verbose = True") :

In [ ]:
jogador42 = Jogador("jogador42",
                  lambda game, state:
                  alphabeta_cutoff_search_new(state,game,depth_for_all,eval_fn=fun_aval_42))

def n_faz_campeonato(num_campeonatos, listaJogadores, nsec=3, verbose=False):
    jogador_vitorias = {}
    print("Progresso com " + str(num_campeonatos) + " campeonatos: ", end='')
    for n_c in range(num_campeonatos):
        campeonato = jogaRastrosNN(listaJogadores, listaJogadores, nsec)
        for jogo in campeonato:
            if verbose:
                print("\nZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ")
                print("CAMPEONATO " + str(n_c + 1))
                mostraJogo(jogo[2][0], verbose = True)
                print(jogo[2])
                print("ZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZ")
        resultado_jogos = [(a,b,n) for (a,b,(x,n)) in campeonato]
        tabela = dict([(jog.nome, 0) for jog in listaJogadores])
        for jogo in resultado_jogos:
            if jogo[2] == 1:
                tabela[jogo[0]] += 1
            else:
                tabela[jogo[1]] += 1
        classificacao = list(tabela.items())
        classificacao.sort(key=lambda p: -p[1])
        for jog in classificacao:
            jogador_vitorias[jog[0]] = jogador_vitorias.get(jog[0],0) + jog[1]
        print(str(n_c + 1) + " ", end='')
    print("\nJOGADOR", "VITÓRIAS")
    for jogador in jogador_vitorias:
        print('{:11}'.format(jogador), '{:>4}'.format(jogador_vitorias[jogador]))
        
todosJog_42 = [jogador42, bacoco, obtusoSW, obtusoNE, arlivre, basilio]
n_faz_campeonato(3, todosJog_42, verbose = False)

Progresso com 3 campeonatos: 

Exemplo da lógica de tentativa de bloqueio do oponente e estilo de "encher" o tabuleiro no lado do oponente.

Os exemplos a seguir são do mesmo jogo, o jogador é o N.

### Outras heurísticas

Para tentar aumentar a taxa de vitória forem feitas outras funções auxiliares que nos testes em torneio não fizeram diferença.

A primeira um algoritmos que deteta se existe um "corredor" para cada jogada que o jogador pode fazer imediatamente que força uma série de jogadas que terminem o jogo e devolve um valor consoante a paridade do número de jogadas:

In [ ]:
#-1 se o jogador perde
#1 se ganha
#0 se não é garantida a vitória ou derrota
def test_forced_path_42(state, player, goal, no_goal, moves):
    def forced_path(move, p_moves):
        if len(p_moves) < 2:
            visited_squares = set()
            visited_squares.add(state.white)
            visited_squares.add(move)
            from_point = None
            while len(p_moves) == 1:
                from_point = p_moves[0]
                p_moves = moves_from_point_42(from_point,visited_squares,state)
                visited_squares.add(from_point)        
            if len(p_moves) == 0:
                if from_point == goal:
                    return 1
                elif from_point == no_goal:
                    return -1
                elif (len(visited_squares) - 1) % 2 == 0:
                    return -1 if state.to_move == player else 1
                elif (len(visited_squares) - 1) % 2 == 1:                
                    return 1 if state.to_move == player else -1
        return 0
    
    has_zero = False
    for move in moves:
        forced_path_res = forced_path(move, moves_from_point_42(move,[state.white], state))
        if forced_path_res == 3:
            return 1
        elif forced_path_res == 0:
            has_zero = True
    if not has_zero:
        return -1 if state.to_move == player else 1
    return 0

A segunda deteta se existe vitória garantida quando a peça branca está num raio de <= 2 da casa objetivo do jogador:

In [ ]:
#Devolve True se vitória for garantida, False se não for garantida.
def test_near_goal_certain_win_42(state, goal):
    def result(sub_state, move):
        blacks = sub_state.blacks.copy()
        blacks.add(sub_state.white)
        return EstadoRastros(to_move=('N' if sub_state.to_move == 'S' else 'S'),white=move,blacks=blacks)    
        
    def retreat(sub_state):
        for move in sub_state.moves():
            if distancia_42(move, goal) >= distancia_42(sub_state.white, goal):
                    if not approach_win(result(sub_state, move)):
                        return True
        return False
    
    def approach_win(sub_state):
        has_approach = False
        for move in sub_state.moves():
            if distancia_42(move, goal) < distancia_42(sub_state.white, goal):
                has_approach = True
                if (move != goal) and retreat(result(sub_state, move)):
                    return False
        return has_approach
    
    return approach_win(state)

A terceira procura verificar se existe uma espaço que impeça a peça branca de chegar aos objetivos e conta a paridade dos espaços livres onde ainda é possível chegar para determinar se o jogador tem vantagem.

É usado o Flood fill.

O algoritmo apesar de conseguir verificar a existencia ou nao da possibilidade de chegar a um dos objetivos, não tem em conta que a paridade dos espaços livres pode não determinar quem tem vantagem pois pode haver estados onde a possível ordem de jogadas não garanta a vantagem quer para o jogador que para o adversário:

In [ ]:
def test_enclosed(state, goal, no_goal, player):
    can_move_to = set()
    def flood_fill(point):
        can_move_to.add(point)
        for x in range (-1,2): 
            for y in range(-1,2):
                point_test = (point[0] + x, point[1] + y)
                if point_test not in state.blacks and point_test not in can_move_to and point_test in state.fullboard:
                    flood_fill(point_test)
                    
    flood_fill(state.white)
    
    if len(can_move_to) == 1:
        return -3 if state.to_move == player else 3
    
    ret = 1 if len(can_move_to) % 2 == 0 else -1
        
    if goal in can_move_to:
        if no_goal in can_move_to:
            return ret
        return 2
    if no_goal in can_move_to:
        return -2
    return ret

Foi gasto algum tempo a implementar estes algoritmos que pensávamos que pudessem ser uteis, mas apesar de nos testes que lhes fizemos funcionarem bem (exceto o último) não aumentaram a taxa de vitória da função de avaliação e sendo assim só serviriam a aumentar o tempo de cada jogada. 